In [1]:
import numpy as np
from scipy.special import expit as sigmoid
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel

# ==========================================================
# 1. Sigmoid + GP 混合 DDM 参数生成器
# ==========================================================
class HybridDDMParameterGenerator:
    def __init__(self, w=0.5):
        """
        w: Sigmoid 与 GP 混合权重 (0~1)
        """
        self.w = w

        # 初始化 GP 模型 —— 分别用于 v, a, t0, z
        kernel = 1.0 * RBF(length_scale=1.0) + WhiteKernel(noise_level=1e-5)
        self.gp_v = GaussianProcessRegressor(kernel=kernel, normalize_y=True)
        self.gp_a = GaussianProcessRegressor(kernel=kernel, normalize_y=True)
        self.gp_t0 = GaussianProcessRegressor(kernel=kernel, normalize_y=True)
        self.gp_z = GaussianProcessRegressor(kernel=kernel, normalize_y=True)

        # Sigmoid 的基础参数
        self.beta_v = np.array([0.01, 0.02, -0.01])
        self.beta_a = np.array([0.005, -0.01, 0.015])
        self.beta_t0 = np.array([-0.001, 0.002, 0.001])
        self.beta_z = np.array([0.02, -0.03, 0.01])

    # ------------------------
    # Sigmoid 基函数
    # ------------------------
    def sigmoid_part(self, X, beta):
        return sigmoid(X @ beta)

    # ------------------------
    # 拟合 GP（需要一批锚点数据）
    # ------------------------
    def fit_gp(self, X, Y_v, Y_a, Y_t0, Y_z):
        self.gp_v.fit(X, Y_v)
        self.gp_a.fit(X, Y_a)
        self.gp_t0.fit(X, Y_t0)
        self.gp_z.fit(X, Y_z)

    # ------------------------
    # 混合预测
    # ------------------------
    def predict_params(self, X):
        """
        X: shape (n,3)  = [P, T, W]
        """
        sig_v = self.sigmoid_part(X, self.beta_v)
        sig_a = self.sigmoid_part(X, self.beta_a)
        sig_t0 = self.sigmoid_part(X, self.beta_t0)
        sig_z = self.sigmoid_part(X, self.beta_z)

        gp_v = self.gp_v.predict(X)
        gp_a = self.gp_a.predict(X)
        gp_t0 = self.gp_t0.predict(X)
        gp_z = self.gp_z.predict(X)

        v = self.w * sig_v + (1 - self.w) * gp_v
        a = self.w * sig_a + (1 - self.w) * gp_a
        t0 = 0.2 + 0.1 * (self.w * sig_t0 + (1 - self.w) * gp_t0)
        z = 0.5 + 0.2 * (self.w * sig_z + (1 - self.w) * gp_z)

        return v, a, t0, z

In [2]:
# ==========================================================
# 2. Euler method: simulate one DDM trial
# ==========================================================
def simulate_ddm_trial(v, a, z, t0, dt=0.001):
    """
    Euler-Maruyama simulation of one DDM trial
    """
    x = z * a
    boundary_top = a
    boundary_bottom = 0
    t = 0

    while True:
        dx = v * dt + np.sqrt(dt) * np.random.randn()
        x += dx
        t += dt

        if x >= boundary_top:
            return 1, t + t0  # upper boundary
        elif x <= boundary_bottom:
            return 0, t + t0  # lower boundary

In [3]:
# ==========================================================
# 3. 生成虚拟反应时与正确率
# ==========================================================
def generate_trials(generator, P, T, W, n=100):
    X = np.array([[P, T, W]])
    v, a, t0, z = generator.predict_params(X)

    v, a, t0, z = float(v), float(a), float(t0), float(z)

    rts = []
    acc = []

    for _ in range(n):
        choice, rt = simulate_ddm_trial(v, a, z, t0)
        acc.append(choice)
        rts.append(rt)

    return np.array(rts), np.array(acc)

In [4]:
# ==========================================================
# 4. Demo：拟合 GP + 生成虚拟数据
# ==========================================================
if __name__ == "__main__":
    # ---- Step 1: 准备锚点数据 ----
    np.random.seed(42)
    X_anchor = np.random.uniform([10, 100, 500], [60, 300, 1500], size=(50, 3))

    # 假设真实值带有噪声
    true_fun = lambda X: np.sin(X[:,0]/50) + np.cos(X[:,1]/80) + 0.0005*X[:,2]

    Y_v = true_fun(X_anchor) + 0.1 * np.random.randn(50)
    Y_a = 1.2 + 0.5 * true_fun(X_anchor) + 0.05 * np.random.randn(50)
    Y_t0 = 0.3 + 0.1 * true_fun(X_anchor) + 0.02 * np.random.randn(50)
    Y_z = 0.5 + 0.1 * true_fun(X_anchor) + 0.02 * np.random.randn(50)

    # ---- Step 2: 初始化混合模型 ----
    gen = HybridDDMParameterGenerator(w=0.6)

    # ---- Step 3: 训练 GP ----
    gen.fit_gp(X_anchor, Y_v, Y_a, Y_t0, Y_z)

    # ---- Step 4: 生成虚拟数据 ----
    rts, acc = generate_trials(gen, P=40, T=200, W=800, n=50)

    print("Mean RT:", np.mean(rts))
    print("Accuracy:", np.mean(acc))


Mean RT: 0.553676126723311
Accuracy: 0.72


c:\Users\蔡振辛\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\gaussian_process\kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\蔡振辛\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\gaussian_process\kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\蔡振辛\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\gaussian_process\kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\User